In [1]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp==2.5.3

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
     |████████████████████████████████| 215.7MB 41kB/s 
     |████████████████████████████████| 204kB 55.8MB/s 
     |████████████████████████████████| 133kB 2.8MB/s 


In [ ]:
# Run this in console to keep Co Lab Running
'''
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
'''

'\nfunction ConnectButton(){\n    console.log("Connect pushed"); \n    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() \n}\nsetInterval(ConnectButton,60000);\n'

In [3]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from pyspark.ml import Pipeline
from pyspark.ml.tuning import TrainValidationSplit
import pandas as pd

In [4]:
# Start Spark session
spark = sparknlp.start()

In [5]:
# Best practice to print versions
print("Spark NLP version", sparknlp.version())
print("Apache Spark version", spark.version)

Spark NLP version 2.5.3
Apache Spark version 2.4.4


In [8]:
#Sample data of only a few rows to test translation code 
url = 'https://raw.githubusercontent.com/Fergus1212/review-star-ranker/jack_nlp/Data/sample_us%20(3).csv'
df_sample = pd.read_csv(url)
sample_df = pd.DataFrame(df_sample[["star_rating","review_body"]])


In [26]:
# Yin genius way of loading Amazon Data
file = 'amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz'
os.system(f'wget https://s3.amazonaws.com/amazon-reviews-pds/tsv/{file}')
df = pd.read_csv(file, compression='gzip', header=0, sep='\t', error_bad_lines=False)

b'Skipping line 1598: expected 15 fields, saw 22\nSkipping line 25768: expected 15 fields, saw 22\n'


In [27]:
df1= df
#headline_updated = df1["review_headline"].str.contains("One Star|Two Stars|Three Stars|Four Stars|Five Stars",na=False)
#df1.drop(headline_updated[headline_updated].index,axis=0,inplace=True)
df1[["star_rating","review_body"]]= df1[["star_rating","review_body"]].astype(str)
df1.star_rating.value_counts()


5    48865
4    13644
1    11034
3     7043
2     5338
Name: star_rating, dtype: int64

In [28]:
df1.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,32114233,R1QX6706ZWJ1P5,B00OYRW4UE,223980852,Elite Sportz Exercise Sliders are Double Sided...,Personal_Care_Appliances,5,0,0,N,Y,Good quality. Shipped,Exactly as described. Good quality. Shipped fast,2015-08-31
1,US,18125776,R3QWMLJHIW6P37,B0000537JQ,819771537,Ezy Dose Weekly,Personal_Care_Appliances,5,0,0,N,Y,Five Stars,It is great,2015-08-31
2,US,19917519,R14Z1VR1N0Z9G6,B00HXXO332,849307176,"Pulse Oximeter, Blood Oxygen Monitor",Personal_Care_Appliances,5,1,1,N,Y,It's really nice it works great,It's really nice it works great. You have the ...,2015-08-31
3,US,18277171,R25ZRJL0GH0U0,B00EOB0JA2,700864740,SE Tools Tool Kit Watch Watch Repair Kit (20 P...,Personal_Care_Appliances,2,0,0,N,Y,Two Stars,The kit works fine... simple cheap plastic tho,2015-08-31
4,US,2593270,R3837KYH7AZNIY,B00OC2O1UC,794298839,"doTERRA HD Clear Facial Kit - Facial Lotion, F...",Personal_Care_Appliances,4,0,1,N,Y,Four Stars,It works better than anything else ive tried,2015-08-31


In [29]:

df_1 = df1.loc[df1["star_rating"]=="1"]
df_2 = df1.loc[df1["star_rating"]=="2"]
df_3 = df1.loc[df1["star_rating"]=="3"]
df_4 = df1.loc[df1["star_rating"]=="4"]
df_5 = df1.loc[df1["star_rating"]=="5"]

df_star1= df_1.sample(n = 5338, replace = True) 
df_star2= df_2.sample(n = 5338, replace = True) 
df_star3= df_3.sample(n = 5338, replace = True) 
df_star4= df_4.sample(n = 5338, replace = True) 
df_star5= df_5.sample(n = 5338, replace = True) 

frames = [df_star1, df_star2, df_star3, df_star4, df_star5]

df_sample = pd.concat(frames)
df_sample[["star_rating","review_body"]]= df[["star_rating","review_body"]].astype(str)

In [6]:
#Only run once, or everytime if using google colab
!pip install googletrans

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 921kB 8.8MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 102kB 9.6MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=97ea0db35f3a4be0cea5e52f40fb832a6506bea938292b5f736e6db16e0a8f72
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=e6e8afc43c58b7de0017f02fa18521fcf1f7185836236d3a98e218a5cae2556a
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars


In [7]:
import googletrans
from googletrans import Translator

In [41]:
translated_df = pd.DataFrame(df_sample[["star_rating","review_body"]])
data = translated_df["review_body"]
source = 'en'
destination = 'fr'

In [42]:
data

63365    Its not good product.Poorly designed.I waned m...
14272    For the money the case should be metal and the...
85346    I knew I was in trouble when it took me about ...
55416    Rarely write reviews but this product was so b...
64432    I bought the finishing touch personal hair rem...
                               ...                        
22278    This works on adults, and children. Seems accu...
39056    This is a big piece of soap that has a very ni...
48877    i admit when i ordered these replacement brush...
52712    The pillow cover fits the Med-Slant pillow per...
18873                                              perfect
Name: review_body, Length: 26690, dtype: object

In [36]:
#Translator function
translator = Translator()
translations = {}

def translator_function(data,source,destination):

  unique_elements = data.unique()
  for element in unique_elements:
        # Adding all the translations to a dictionary (translations)
    translations[element] = translator.translate(element, src = source, dest=destination).text
        #translated_df = pd.DataFrame(translations)

In [43]:
source = 'en'
destination = 'fr'
translator_function(data,source,destination)

In [45]:
en_fr = translations 

In [46]:
# Replacing all the translated words from the dictionary to the original dataframe
translated_df.replace(en_fr, inplace = True)
translated_df.head(10)

,star_rating,review_body
63365,1,Ce n'est pas un bon produit.Malement conçu.J'a...
14272,1,"Pour l'argent, le boîtier doit être en métal e..."
85346,1,Je savais que j'avais des problèmes quand il m...
55416,1,Écrivez rarement des critiques mais ce produit...
64432,1,J'ai acheté l'épilateur personnel de finition ...
592,1,J'étais tellement excité par ce produit mais j...
63063,1,J'en ai acheté un dans un magasin de sport et ...
15761,1,Mauvais. Ça ne marche pas
34339,1,Il s'est cassé après la deuxième utilisation ....
67240,1,"Regardez les ingrédients, c'est de l'eau sucré..."


In [47]:
#Now translate back again
source = 'fr'
destination = 'en'
data = translated_df["review_body"]
translator_function(data,source,destination)

In [51]:
translated_df.replace(en_fr, inplace = True)
translated_df.head()

,star_rating,review_body
63365,1,It is not a good product. Badly designed. I ha...
14272,1,"For the money, the case must be made of metal ..."
85346,1,I knew I had problems when it took me about 20...
55416,1,Rarely write reviews but this product was so b...
64432,1,I bought the personal finishing epilator from ...


In [52]:
# Select out features and labels
entireDataset = spark.createDataFrame(translated_df[['star_rating', 'review_body']])

In [53]:
# Check data types
entireDataset.dtypes

[('star_rating', 'string'), ('review_body', 'string')]

In [54]:
# View col names
entireDataset.columns

['star_rating', 'review_body']

In [55]:
# Change object to string
entireDataset.review_body = entireDataset.review_body.astype('string')

In [56]:
# Prepare for NLP pipeline with initial DocumentAssembler
# Content is inside review_body
document = DocumentAssembler() \
  .setInputCol("review_body") \
  .setOutputCol("document")

In [57]:
# Using sentence detector (pretrained)
# Sentence detector
use = UniversalSentenceEncoder.pretrained() \
  .setInputCols(["document"]) \
  . setOutputCol("sentence_embeddings")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [58]:
# Our labels / classes are the 1 - 5 star ratings
# These are in the star_rating col

classifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("star_rating")\
  .setMaxEpochs(15)\
  .setEnableOutputLogs(True)

In [59]:
# Declare the pipeline
use_clf_pipeline = Pipeline(
  stages = [
    document,
    use,
    classifierdl
  ]
)



In [60]:
# Split into training and testing data
trainDataset, testDataset = entireDataset.randomSplit([0.75, 0.25], seed=12345)


In [61]:
# Begin fitting on the data
use_pipelineModel = use_clf_pipeline.fit(trainDataset)

In [62]:
# Set predictions on testDataset
predictions = use_pipelineModel.transform(testDataset)
predictions.count()
predictions.show()

+-----------+--------------------+--------------------+--------------------+--------------------+
|star_rating|         review_body|            document| sentence_embeddings|               class|
+-----------+--------------------+--------------------+--------------------+--------------------+
|          1|(Health and beaut...|[[document, 0, 23...|[[sentence_embedd...|[[category, 0, 23...|
|          1|* Good point of v...|[[document, 0, 17...|[[sentence_embedd...|[[category, 0, 17...|
|          1|*** Edited since ...|[[document, 0, 66...|[[sentence_embedd...|[[category, 0, 66...|
|          1|1 star is too hig...|[[document, 0, 30...|[[sentence_embedd...|[[category, 0, 30...|
|          1|1) The sound leve...|[[document, 0, 92...|[[sentence_embedd...|[[category, 0, 92...|
|          1|1.  It does NOT h...|[[document, 0, 60...|[[sentence_embedd...|[[category, 0, 60...|
|          1|A comparison of t...|[[document, 0, 38...|[[sentence_embedd...|[[category, 0, 38...|
|          1|A few w

In [63]:

# Run predictions
predictions.select('star_rating','review_body', 'class.result').show(10, truncate=80)

+-----------+--------------------------------------------------------------------------------+------+
|star_rating|                                                                     review_body|result|
+-----------+--------------------------------------------------------------------------------+------+
|          1|(Health and beauty) <br /> The product was delivered on time, but the product...|   [1]|
|          1|* Good point of views; ^^ * 1. The design is perfect to use this on the bed, ...|   [2]|
|          1|*** Edited since 1 star review was published *** I had some issues with recei...|   [1]|
|          1|1 star is too high for this product. What else can I say, it just doesn't cut...|   [1]|
|          1|1) The sound level does not compare to my warm mist humidifier which broke wh...|   [2]|
|          1|1.  It does NOT heat up quickly<br />2.  I left the rollers in for a long tim...|   [1]|
|          1|A comparison of three different maternity support belts    At 22 week

In [64]:
# Import reports for quantifying results
from sklearn.metrics import classification_report, accuracy_score

df = use_pipelineModel.transform(testDataset).select('star_rating','review_body', 'class.result').toPandas()

df['result'] = df['result'].apply(lambda x: x[0])

In [65]:
df.result = df.result.astype(str)

In [66]:
print(classification_report(df['star_rating'], df.result))
print(accuracy_score(df['star_rating'], df.result))

              precision    recall  f1-score   support

           1       0.53      0.62      0.57      1363
           2       0.36      0.36      0.36      1321
           3       0.39      0.33      0.35      1329
           4       0.40      0.31      0.35      1368
           5       0.52      0.64      0.57      1335

    accuracy                           0.45      6716
   macro avg       0.44      0.45      0.44      6716
weighted avg       0.44      0.45      0.44      6716

0.45131030375223347
